### Hyperparameter Tuning of a Neural Network Model

In [1]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

In [2]:
# Generate a sample dataset
X, y = make_classification(n_samples = 1000, n_features = 20, n_classes = 2, random_state = 42)

In [3]:
# Preprocess the data by scaling the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
# Define the model-building function
def create_model(n_layers = 1, n_units = 32, dropout_rate = 0.2, learning_rate = 0.001):

    model = Sequential()

    # Set the first hidden layer
    model.add(Dense(n_units, activation = 'relu', input_shape = (20,)))
    model.add(Dropout(dropout_rate))
    
    # Set additional hidden layers
    for _ in range(n_layers - 1):
        model.add(Dense(n_units, activation = 'relu'))
        model.add(Dropout(dropout_rate))
    
    # Set the output layer
    model.add(Dense(1, activation = 'sigmoid'))
    
    # Define the optimizer
    optimizer = Adam(learning_rate = learning_rate)

    # Compile the model
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [5]:
# Create the KerasClassifier
model = KerasClassifier(model = create_model, verbose = 0)

In [6]:
# Define the hyperparameter search space
search_space = {'model__n_layers': [1, 2, 3],
                'model__n_units': [16, 32, 64, 128],
                'model__dropout_rate': [0.1, 0.2, 0.3, 0.4],
                'model__learning_rate': [0.001, 0.01, 0.1],
                'batch_size': [16, 32, 64],
                'epochs': [50, 100, 150]}

In [7]:
# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator = model,
                                   param_distributions = search_space,
                                   n_iter = 40,
                                   cv = 3,
                                   verbose = 2,
                                   n_jobs = -1,
                                   random_state = 42)

# Perform the random search and fit the model
random_search_result = random_search.fit(X, y)

# Print the best hyperparameters and the best cross-validation score
print("Best Parameters:", random_search_result.best_params_)
print("Best Cross-Validation Score:", random_search_result.best_score_)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


c:\Users\wodnj\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Parameters: {'model__n_units': 16, 'model__n_layers': 2, 'model__learning_rate': 0.001, 'model__dropout_rate': 0.3, 'epochs': 50, 'batch_size': 16}
Best Cross-Validation Score: 0.8699777621933311


In [8]:
# Evaluate the best model on the entire dataset
best_model = random_search_result.best_estimator_
score = best_model.score(X, y)
print("Accuracy from the Best Model:", best_model.score(X, y))

Accuracy from the Best Model: 0.906
